In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import glob
import os
import re
from pynwb import NWBFile, TimeSeries, NWBHDF5IO
from IPython.display import display, HTML


from aind_dynamic_foraging_data_utils import nwb_utils as nu
import aind_dynamic_foraging_basic_analysis.licks.annotation as a

from aind_dynamic_foraging_basic_analysis import plot_foraging_session, compute_foraging_efficiency

In [3]:
# Load single NWB

nwb = nu.load_nwb_from_filename('/root/capsule/data/foraging_nwb_bonsai/687582_2023-11-16_12-13-24.nwb')

/opt/conda/lib/python3.9/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.5.1 because version 1.8.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/opt/conda/lib/python3.9/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'core' version 2.4.0 because version 2.7.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/opt/conda/lib/python3.9/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-experimental' version 0.2.0 because version 0.5.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


In [41]:
nwb

,start_time,stop_time,animal_response,rewarded_historyL,rewarded_historyR,delay_start_time,goCue_start_time,reward_outcome_time,bait_left,bait_right,base_reward_probability_sum,reward_probabilityL,reward_probabilityR,reward_random_number_left,reward_random_number_right,left_valve_open_time,right_valve_open_time,block_beta,block_min,block_max,min_reward_each_block,delay_beta,delay_min,delay_max,delay_duration,ITI_beta,ITI_min,ITI_max,ITI_duration,response_duration,reward_consumption_duration,auto_waterL,auto_waterR,laser_on_trial,laser_wavelength,laser_location,laser_1_power,laser_2_power,laser_on_probability,laser_duration,laser_condition,laser_condition_probability,laser_start,laser_start_offset,laser_end,laser_end_offset,laser_protocol,laser_frequency,laser_rampingdown,laser_pulse_duration,session_wide_control,fraction_of_session,session_start_with,session_alternation,auto_train_engaged,auto_train_curriculum_name,auto_train_curriculum_version,auto_train_curriculum_schema_version,auto_train_stage,auto_train_stage_overridden,lickspout_position_x,lickspout_position_y,lickspout_position_z,reward_size_left,reward_size_right
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,4.489577e+06,4.489585e+06,2.0,False,False,4.489578e+06,4.489579e+06,4.489582e+06,False,False,0.8,0.711111,0.088889,0.963379,0.207240,0.042,0.045,20.0,20.0,40.0,1.0,0.5,1.0,1.5,1.004837,2.0,1.0,7.0,1.092118,3.0,3.0,0,0,0,NaN,None,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN,None,NaN,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,4.0,4.0
1,4.489585e+06,4.489594e+06,2.0,False,False,4.489587e+06,4.489588e+06,4.489591e+06,True,False,0.8,0.711111,0.088889,0.526161,0.362175,0.042,0.045,20.0,20.0,40.0,1.0,0.5,1.0,1.5,1.500000,2.0,1.0,7.0,1.675733,3.0,3.0,0,0,0,NaN,None,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN,None,NaN,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,4.0,4.0
2,4.489594e+06,4.489605e+06,2.0,False,False,4.489597e+06,4.489599e+06,4.489602e+06,True,False,0.8,0.711111,0.088889,0.904736,0.315749,0.042,0.045,20.0,20.0,40.0,1.0,0.5,1.0,1.5,1.500000,2.0,1.0,7.0,3.099859,3.0,3.0,0,0,0,NaN,None,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN,None,NaN,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,4.0,4.0
3,4.489605e+06,4.489616e+06,2.0,False,False,4.489609e+06,4.489610e+06,4.489613e+06,True,False,0.8,0.711111,0.088889,0.758493,0.959065,0.042,0.045,20.0,20.0,40.0,1.0,0.5,1.0,1.5,1.024207,2.0,1.0,7.0,3.777559,3.0,3.0,0,0,0,NaN,None,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN,None,NaN,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,4.0,4.0
,start_time,stop_time,animal_response,rewarded_historyL,rewarded_historyR,delay_start_time,goCue_start_time,reward_outcome_time,bait_left,bait_right,base_reward_probability_sum,reward_probabilityL,reward_probabilityR,reward_random_number_left,reward_random_number_right,left_valve_open_time,right_valve_open_time,block_beta,block_min,block_max,min_reward_each_block,delay_beta,delay_min,delay_max,delay_duration,ITI_beta,ITI_min,ITI_max,ITI_duration,response_duration,reward_consumption_duration,auto_waterL,auto_waterR,laser_on_trial,laser_wavelength,laser_location,laser_1_power,laser_2_power,laser_on_probability,laser_duration,laser_condition,laser_condition_probability,laser_start,laser_start_offset,laser_end,laser_end_offset,laser_protocol,laser_frequency,laser_rampingdown,laser_pulse_duration,session_wide_control,fraction_of_session,session_start_with,session_alternation,auto_train_engaged,auto_train_curriculum_name,auto_train_curriculum_version,auto_train_curriculum_schema_version,auto_train_stage,auto_train_stage_overridden,lickspout_position_x,lickspout_position_y,lickspout_position_z,reward_size_left,reward_size_right
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,4.489577e+06,4.489585e+06,2.0,False,False,4.489578e+06,4.489579e+06,4.489582e+06,False,False,0.8,0.711111,0.088889,0.963379,0.207240,0.042,0.045,20.0,2

In [42]:
print(type(nwb.trials))  # Check if it's a tuple or a container
print(nwb.trials)  # Inspect the contents to find where the table is

<class 'pynwb.epoch.TimeIntervals'>
trials pynwb.epoch.TimeIntervals at 0x139823426858816
Fields:
  colnames: ['start_time' 'stop_time' 'animal_response' 'rewarded_historyL'
 'rewarded_historyR' 'delay_start_time' 'goCue_start_time'
 'reward_outcome_time' 'bait_left' 'bait_right'
 'base_reward_probability_sum' 'reward_probabilityL' 'reward_probabilityR'
 'reward_random_number_left' 'reward_random_number_right'
 'left_valve_open_time' 'right_valve_open_time' 'block_beta' 'block_min'
 'block_max' 'min_reward_each_block' 'delay_beta' 'delay_min' 'delay_max'
 'delay_duration' 'ITI_beta' 'ITI_min' 'ITI_max' 'ITI_duration'
 'response_duration' 'reward_consumption_duration' 'auto_waterL'
 'auto_waterR' 'laser_on_trial' 'laser_wavelength' 'laser_location'
 'laser_1_power' 'laser_2_power' 'laser_on_probability' 'laser_duration'
 'laser_condition' 'laser_condition_probability' 'laser_start'
 'laser_start_offset' 'laser_end' 'laser_end_offset' 'laser_protocol'
 'laser_frequency' 'laser_rampingdow

In [31]:
# Create df_trial

nwb.df_trials = nu.create_df_trials(nwb)

Timestamps are adjusted such that `_in_session` timestamps start at the first go cue


/opt/conda/lib/python3.9/site-packages/aind_dynamic_foraging_data_utils/nwb_utils.py:457: UserWarning: Reward before choice time. This is likely due to manual rewards
  warnings.warn("Reward before choice time. This is likely due to manual rewards")


In [32]:
nwb.df_trials.columns

Index(['trial', 'animal_response', 'rewarded_historyL', 'rewarded_historyR',
       'bait_left', 'bait_right', 'base_reward_probability_sum',
       'reward_probabilityL', 'reward_probabilityR',
       'reward_random_number_left', 'reward_random_number_right',
       'left_valve_open_time', 'right_valve_open_time', 'block_beta',
       'block_min', 'block_max', 'min_reward_each_block', 'delay_beta',
       'delay_min', 'delay_max', 'delay_duration', 'ITI_beta', 'ITI_min',
       'ITI_max', 'ITI_duration', 'response_duration',
       'reward_consumption_duration', 'auto_waterL', 'auto_waterR',
       'laser_on_trial', 'laser_wavelength', 'laser_location', 'laser_1_power',
       'laser_2_power', 'laser_on_probability', 'laser_duration',
       'laser_condition', 'laser_condition_probability', 'laser_start',
       'laser_start_offset', 'laser_end', 'laser_end_offset', 'laser_protocol',
       'laser_frequency', 'laser_rampingdown', 'laser_pulse_duration',
       'session_wide_control', 

In [43]:
LEFT, RIGHT, IGNORE = 0, 1, 2

In [46]:
import numpy as np
import pandas as pd

def add_compute_df_trial_metrics(df_trials, nwb, LEFT=0, RIGHT=1, IGNORE=2):
    """
    Adds metrics from compute_df_trial to an existing df_trials DataFrame.
    
    Parameters:
    -----------
    df_trials : pandas.DataFrame
        DataFrame created by create_df_trials function
    nwb : NWB object
        The original NWB file object
    LEFT : int, optional
        Value representing left choice (default 0)
    RIGHT : int, optional
        Value representing right choice (default 1)
    IGNORE : int, optional
        Value representing ignored trial (default 2)
    
    Returns:
    --------
    pandas.DataFrame
        Updated DataFrame with additional metrics
    """
    # Make a copy to avoid modifying the original DataFrame
    df = df_trials.copy()
    
    # --- Add reward-related columns ---
    df['reward'] = False
    df.loc[(df.rewarded_historyL | df.rewarded_historyR
            | df.auto_waterR | df.auto_waterL 
            & (df.animal_response != IGNORE)) > 0, 'reward'] = True
    
    df['reward_non_autowater'] = False
    df.loc[(df.rewarded_historyL | df.rewarded_historyR), 'reward_non_autowater'] = True
    
    df['non_autowater_trial'] = False
    df.loc[(df.auto_waterL==0) & (df.auto_waterR==0), 'non_autowater_trial'] = True
    
    df['non_autowater_finished_trial'] = df['non_autowater_trial'] & (df['animal_response'] != IGNORE)
    df['ignored_non_autowater'] = df['non_autowater_trial'] & (df['animal_response'] == IGNORE)
    df['ignored_autowater'] = ~df['non_autowater_trial'] & (df['animal_response'] == IGNORE)
    
    # --- Lick-related stats ---    
    all_left_licks = nwb.acquisition['left_lick_time'].timestamps[:]
    all_right_licks = nwb.acquisition['right_lick_time'].timestamps[:]
    
    # Define the start and stop time for each epoch
    # Use _in_session columns to ensure we're using the adjusted times
    lick_stats_epochs = {
        'gocue_stop': ['goCue_start_time_in_session', 'stop_time_in_session'],
        'delay_period': ['delay_start_time_in_session', 'goCue_start_time_in_session'],
        'iti': ['start_time_in_session', 'delay_start_time_in_session'],        
    }
    
    # Trial-by-trial counts
    for i in range(len(df)):
        for epoch_name, (start_time_name, stop_time_name) in lick_stats_epochs.items():
            start_time, stop_time = df.loc[i, [start_time_name, stop_time_name]]
            
            # Lick analysis for the specific epoch
            left_licks = all_left_licks[(all_left_licks > start_time) & (all_left_licks < stop_time)]
            right_licks = all_right_licks[(all_right_licks > start_time) & (all_right_licks < stop_time)]
            all_licks = np.hstack([left_licks, right_licks])
            
            # Lick counts
            df.loc[i, f'duration_{epoch_name}'] = stop_time - start_time
            df.loc[i, f'n_lick_left_{epoch_name}'] = len(left_licks)
            df.loc[i, f'n_lick_right_{epoch_name}'] = len(right_licks)
            df.loc[i, f'n_lick_all_{epoch_name}'] = len(all_licks)
            
            # Lick switches
            if len(all_licks) > 1:
                _lick_identity = np.hstack([np.ones(len(left_licks)) * LEFT, np.ones(len(right_licks)) * RIGHT])
                _lick_identity_sorted = [x for x, _ in sorted(zip(_lick_identity, all_licks), key=lambda pairs: pairs[1])]
                df.loc[i, f'n_lick_switches_{epoch_name}'] = np.sum(np.diff(_lick_identity_sorted) != 0)
                
                # Lick consistency
                choice = df.loc[i, 'animal_response']
                df.loc[i, f'n_lick_consistency_{epoch_name}'] = (
                    np.sum(_lick_identity_sorted == choice) / len(_lick_identity_sorted)
                    if len(_lick_identity_sorted) > 0 else np.nan
                )
            else:
                df.loc[i, f'n_lick_switches_{epoch_name}'] = 0
                df.loc[i, f'n_lick_consistency_{epoch_name}'] = np.nan
            
            # Special treatment for gocue to stop epoch
            if epoch_name == 'gocue_stop':
                # Reaction time
                first_lick = all_licks.min() if len(all_licks) > 0 else np.nan
                df.loc[i, 'reaction_time'] = (
                    first_lick - df.loc[i, 'goCue_start_time_in_session']
                    if not np.isnan(first_lick) else np.nan
                )
                
                # Handle ignored trials
                if df.loc[i, 'animal_response'] == IGNORE:
                    df.loc[i, 'reaction_time'] = np.nan
                    df.loc[i, 'n_valid_licks_left'] = 0
                    df.loc[i, 'n_valid_licks_right'] = 0
                    df.loc[i, 'n_valid_licks_all'] = 0
    
    return df

In [49]:
nwb.df_trials = add_compute_df_trial_metrics(df_trials, nwb)

In [50]:
nwb.df_trials

,trial,animal_response,rewarded_historyL,rewarded_historyR,bait_left,bait_right,base_reward_probability_sum,reward_probabilityL,reward_probabilityR,reward_random_number_left,...,n_lick_right_delay_period,n_lick_all_delay_period,n_lick_switches_delay_period,n_lick_consistency_delay_period,duration_iti,n_lick_left_iti,n_lick_right_iti,n_lick_all_iti,n_lick_switches_iti,n_lick_consistency_iti
0,0,2.0,False,False,False,False,0.8,0.711111,0.088889,0.963379,...,0.0,0.0,0.0,NaN,1.112992,0.0,0.0,0.0,0.0,NaN
1,1,2.0,False,False,True,False,0.8,0.711111,0.088889,0.526161,...,0.0,0.0,0.0,NaN,1.679008,0.0,0.0,0.0,0.0,NaN
2,2,2.0,False,False,True,False,0.8,0.711111,0.088889,0.904736,...,0.0,0.0,0.0,NaN,3.105984,0.0,0.0,0.0,0.0,NaN
3,3,2.0,False,False,True,False,0.8,0.711111,0.088889,0.758493,...,0.0,0.0,0.0,NaN,3.774016,0.0,0.0,0.0,0.0,NaN
4,4,2.0,False,False,True,False,0.8,0.711111,0.088889,0.555699,...,0.0,0.0,0.0,NaN,4.756000,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,537,2.0,False,False,True,False,0.8,0.711111,0.088889,0.608060,...,0.0,0.0,0.0,NaN,5.978016,0.0,0.0,0.0,0.0,NaN
538,538,2.0,False,False,True,False,0.8,0.711111,0.088889,0.030398,...,0.0,0.0,0.0,NaN,3.545984,0.0,0.0,0.0,0.0,NaN
539,539,2.0,False,False,True,False,0.8,0.711111,0.088889,0.726525,...,0.0,0.0,0.0,NaN,4.328000,0.0,0.0,0.0,0.0,NaN
540,540,2.0,False,False,True,True,0.8,0.711111,0.088889,0.587486,...,0.0,0.0,0.0,NaN,1.628000,0.0,0.0,0.0,0.0,NaN


In [51]:
def session_metrics(nwb):
    """ 
    Compute all session level metrics

    Includes session level metadata as a temporary organizer 
    Includes majority of metrics from process_nwbs.py

    New addition: chosen_probability - average difference between the chosen probability
    and non-chosen probability / the difference between the largest and smallest probability 
    in the session
    """

    if not hasattr(nwb, 'df_trials'):
        print('You need to compute df_trials: nwb_utils.create_trials_df(nwb)')
        return
    
    df = nwb.df_trials.copy()

    # METADATA PLACEHOLDER 
    session_start_time = nwb.session_start_time
    session_date = session_start_time.strftime("%Y-%m-%d")
    subject_id_from_meta = nwb.subject.subject_id
    
    # Parse the file name for suffix
    old_re = re.match(r"(?P<subject_id>\d+)_(?P<date>\d{4}-\d{2}-\d{2})(?:_(?P<n>\d+))?\.json", 
                nwb.session_id)
    
    if old_re is not None:
        subject_id, session_date, nwb_suffix = old_re.groups()
        nwb_suffix = int(nwb_suffix) if nwb_suffix is not None else 0
    else:
        subject_id, session_date, session_json_time = re.match(r"(?P<subject_id>\d+)_(?P<date>\d{4}-\d{2}-\d{2})(?:_(?P<time>.*))\.json", 
                            nwb.session_id).groups()
        nwb_suffix = int(session_json_time.replace('-', ''))
    
    # Verify metadata matches
    assert subject_id == subject_id_from_meta, f"Subject name from the metadata ({subject_id_from_meta}) does not match "\
                                               f"that from json name ({subject_id})!!"
    assert session_date == session_date, f"Session date from the metadata ({session_date}) does not match "\
                                                   f"that from json name ({session_date})!!"
    
    # Create session index
    session_index = pd.MultiIndex.from_tuples([(subject_id, session_date, nwb_suffix)], 
                                            names=['subject_id', 'session_date', 'nwb_suffix'])

    # Get metadata from nwb.scratch
    meta_dict = nwb.scratch['metadata'].to_dataframe().iloc[0].to_dict()
    
    # Calculate performance metrics
    n_total_trials = len(df)
    n_finished_trials = (df.animal_response != IGNORE).sum()
    
    # Actual foraging trials (autowater excluded)
    n_total_trials_non_autowater = df.non_autowater_trial.sum()
    n_finished_trials_non_autowater = df.non_autowater_finished_trial.sum()
    
    n_reward_trials_non_autowater = df.reward_non_autowater.sum()
    reward_rate_non_autowater_finished = n_reward_trials_non_autowater / n_finished_trials_non_autowater if n_finished_trials_non_autowater > 0 else np.nan
    
    # Calculate foraging efficiency
    foraging_eff, foraging_eff_random_seed = compute_foraging_efficiency(
        baited='without bait' not in nwb.protocol.lower(),
        choice_history=df.animal_response.map({0: 0, 1: 1, 2: np.nan}).values,
        reward_history=df.rewarded_historyL | df.rewarded_historyR,
        p_reward=[
            df.reward_probabilityL.values,
            df.reward_probabilityR.values,
        ],
        random_number=[
            df.reward_random_number_left.values,
            df.reward_random_number_right.values,
        ],
        autowater_offered=(df.auto_waterL == 1) | (df.auto_waterR == 1),
    )
    
    # Override foraging_eff_random_seed for old bpod sessions
    if 'bpod' in nwb.session_description:
        foraging_eff_random_seed = nwb.get_scratch('metadata')['foraging_efficiency_with_actual_random_seed'].values[0]
    
    finished_rate = n_finished_trials_non_autowater / n_total_trials_non_autowater if n_total_trials_non_autowater > 0 else np.nan
    
    # New Metrics

    # Probability chosen calculation 
    probability_chosen = []
    probability_not_chosen = []

    for _, row in df.iterrows():
        if row.animal_response == 2:
            probability_chosen.append(np.nan)
            probability_not_chosen.append(np.nan)
        elif row.animal_response == 0:  # Chosen = left choice left probability, not chosen = left choice right probability 
            probability_chosen.append(row.reward_probabilityL)
            probability_not_chosen.append(row.reward_probabilityR)
        else: # Chosen = right choice right probability, not chosen = right choice left probability 
            probability_chosen.append(row.reward_probabilityR)
            probability_not_chosen.append(row.reward_probabilityL)

    df['probability_chosen'] = probability_chosen
    df['probability_not_chosen'] = probability_not_chosen

    # Calculate the chosen probability
    average = (df['probability_chosen'] - df['probability_not_chosen'])
    
    p_larger_global = max(df['probability_chosen'].max(), df['probability_not_chosen'].max())
    
    p_smaller_global = min(df['probability_chosen'].min(), df['probability_not_chosen'].min())
    
    mean_difference = average.mean()
    chosen_probability = mean_difference / (p_larger_global - p_smaller_global)


    # Pack all data
    dict_meta = {
        # Basic metadata
        'rig': meta_dict['box'],        
        'user_name': nwb.experimenter[0],
        'task': nwb.protocol,
        
        # New metric
        'chosen_probability': chosen_probability,

        # Trial counts and rates
        'total_trials': n_total_trials_non_autowater,
        'finished_trials': n_finished_trials_non_autowater,
        'finished_rate': finished_rate,
        'total_trials_with_autowater': n_total_trials,
        'finished_trials_with_autowater': n_finished_trials,
        'finished_rate_with_autowater': n_finished_trials / n_total_trials,
        
        # Reward and foraging metrics
        'reward_trials': n_reward_trials_non_autowater,
        'reward_rate': reward_rate_non_autowater_finished,
        'foraging_eff': foraging_eff,
        'foraging_eff_random_seed': foraging_eff_random_seed,
        'foraging_performance': foraging_eff * finished_rate,
        'foraging_performance_random_seed': foraging_eff_random_seed * finished_rate,
        
        # Timing metrics
        'reaction_time_median': df.loc[:, 'reaction_time'].median(),
        'reaction_time_mean': df.loc[:, 'reaction_time'].mean(),
        'early_lick_rate': (df.loc[:, 'n_lick_all_delay_period'] > 0).sum() / n_total_trials,
        
        # Double dipping metrics
        'double_dipping_rate_finished_trials': 
            (df.loc[(df.animal_response != IGNORE), 'n_lick_switches_gocue_stop'] > 0).sum() 
            / (df.animal_response != IGNORE).sum(),
        'double_dipping_rate_finished_reward_trials':
            (df.loc[df.reward, 'n_lick_switches_gocue_stop'] > 0).sum()  
            / df.reward.sum(),
        'double_dipping_rate_finished_noreward_trials':
            (df.loc[(df.animal_response != IGNORE) & (~df.reward), 'n_lick_switches_gocue_stop'] > 0).sum() 
            / ((df.animal_response != IGNORE) & (~df.reward)).sum(),
            
        # Lick consistency metrics
        'lick_consistency_mean_finished_trials': 
            df.loc[(df.animal_response != IGNORE), 'n_lick_consistency_gocue_stop'].mean(),
        'lick_consistency_mean_finished_reward_trials': 
            df.loc[df.reward, 'n_lick_consistency_gocue_stop'].mean(),
        'lick_consistency_mean_finished_noreward_trials': 
            df.loc[(df.animal_response != IGNORE) & (~df.reward), 'n_lick_consistency_gocue_stop'].mean(),
    }
    
    # Create DataFrame with hierarchical columns
    df_meta = pd.DataFrame(dict_meta, 
                          index=session_index)
    df_meta.columns = pd.MultiIndex.from_product([['metadata'], dict_meta.keys()],
                                                names=['type', 'variable'])
    
    return df_meta

In [52]:
final_df = session_metrics(nwb)

In [53]:
final_df

type                               metadata                                 \
variable                                rig     user_name             task   
subject_id session_date nwb_suffix                                           
687582     2023-11-16   121324      Green-1  Avalon Amaya  Coupled Baiting   

type                                                                \
variable                           chosen_probability total_trials   
subject_id session_date nwb_suffix                                   
687582     2023-11-16   121324               0.170124          484   

type                                                              \
variable                           finished_trials finished_rate   
subject_id session_date nwb_suffix                                 
687582     2023-11-16   121324                 202      0.417355   

type                                                            \
variable                           total_trials_with_autowater   
subject_id session_date nwb_suffix                               
687582     2023-11-16   121324                             542   

type                                                               \
variable                           finished_trials_with_autowater   
subject_id session_date nwb_suffix                                  
687582     2023-11-16   121324                                241   

type                                                             ...  \
variable                           finished_rate_with_autowater  ...   
subject_id session_date nwb_suffix                               ...   
687582     2023-11-16   121324                         0.444649  ...   

type                                                                 \
variable                           foraging_performance_random_seed   
subject_id session_date nwb_suffix                                    
687582     2023-11-16   121324                             0.303531   

type                                                                        \
variable                           reaction_time_median reaction_time_mean   
subject_id session_date nwb_suffix                                           
687582     2023-11-16   121324                      NaN                NaN   

type                                                \
variable                           early_lick_rate   
subject_id session_date nwb_suffix                   
687582     2023-11-16   121324                 0.0   

type                                                                    \
variable                           double_dipping_rate_finished_trials   
subject_id session_date nwb_suffix                                       
687582     2023-11-16   121324                                     0.0   

type                                                                           \
variable                           double_dipping_rate_finished_reward_trials   
subject_id session_date nwb_suffix                                              
687582     2023-11-16   121324                                            0.0   

type                                                                             \
variable                           double_dipping_rate_finished_noreward_trials   
subject_id session_date nwb_suffix                                                
687582     2023-11-16   121324                                              0.0   

type                                                                      \
variable                           lick_consistency_mean_finished_trials   
subject_id session_date nwb_suffix                                         
687582     2023-11-16   121324                                       NaN   

type                                                                             \
variable                           lick_consistency_mean_finished_reward_trials   
subject_id session_date nwb_suffix                           